In [88]:
import ppi_py
import pandas as pd
import numpy as np
import PythonFunctions as pf
from scipy import stats

rhos = pd.read_csv("../Data/5_rho.csv")
rhos

,y,x,beta_ppi,beta_hum,beta_sil,beta_sil_sm,se_ppi,se_hum,se_sil,lower_ppi,upper_ppi,lower_hum,upper_hum,lower_sil,upper_sil,ppi_corr,n,N
0,gpt4turbo_wp_Saved,Intervention,0.081381,0.081581,0.081985,0.081985,0.001163,0.001173,0.002817,0.079101,0.083660,0.079282,0.083881,0.076464,0.087507,0.352677,1000000,163962
1,gpt4turbo_wp_Saved,Barrier,0.105810,0.105930,0.484486,0.484486,0.001757,0.001768,0.003702,0.102366,0.109254,0.102464,0.109396,0.477231,0.491741,0.314138,1000000,163962
2,gpt4turbo_wp_Saved,Gender,0.134010,0.134204,0.197077,0.197077,0.002694,0.002707,0.006378,0.128728,0.139289,0.128897,0.139510,0.184577,0.209576,0.262760,1000000,163962
3,gpt4turbo_wp_Saved,Fitness,0.174661,0.174951,0.016098,0.016098,0.002823,0.002839,0.006787,0.169130,0.180196,0.169386,0.180516,0.002797,0.029399,0.283120,1000000,163962
4,gpt4turbo_wp_Saved,Social Status,0.248259,0.248827,0.026267,0.026267,0.007595,0.007623,0.018429,0.233386,0.263158,0.233887,0.263767,-0.009853,0.062387,0.229833,1000000,163962
5,gpt4turbo_wp_Saved,CrossingSignal,0.378954,0.378903,0.657856,0.657856,0.001954,0.001967,0.003958,0.375125,0.382783,0.375048,0.382759,0.650100,0.665613,0.308987,1000000,163962
6,gpt4turbo_wp_Saved,Age,0.506392,0.506359,0.182285,0.182285,0.002440,0.002446,0.006541,0.501611,0.511174,0.501564,0.511154,0.169465,0.195105,0.198339,1000000,163962
7,gpt4turbo_wp_Saved,Utilitarian,0.572248,0.572312,0.552874,0.552874,0.002256,0.002262,0.005516,0.567826,0.576670,0.567878,0.576746,0.542063,0.563686,0.191972,1000000,163962
8,gpt4turbo_wp_Saved,Species,0.684679,0.684791,0.833830,0.833830,0.002011,0.002011,0.003904,0.680738,0.688620,0.680849,0.688733,0.826178,0.841482,0.049155,1000000,163962


### Compute effective sample sizes

In [81]:
# function to compute effective sample size
def n0 (n, N, rho):
    
    n0 = (n*(n+N)) / (n+(N*(1-rho**2)))

    rounded = int(round(n0, 0))

    return rounded


In [108]:
# Define labels for attributes
labels = pd.DataFrame({
    "Attribute": [
        "Species",
        "Social Status",
        "Utilitarian",
        "Age",
        "Gender",
        "Fitness",
        "CrossingSignal",
        "Barrier",
        "Intervention"
    ],
    "Label": [
        "Sparing humans vs animals",
        "Sparing high status vs low status",
        "Sparing more characters vs fewer",
        "Sparing the young vs old",
        "Sparing women vs men",
        "Sparing the fit vs the large",
        "Sparing the lawful vs unlawful",
        "Sparing pedestrians vs passengers",
        "Preference for inaction vs intervention"
    ]
})

# Compute effective sample size for each attribute
df = pd.DataFrame({
    "x": rhos["x"],
    "ppi_corr": round(rhos["ppi_corr"],3),
    "n0": rhos["ppi_corr"].apply(lambda rho: n0(n=10**4, N=10**5, rho=rho))
})

# Merge labels and effective sample size
joined_df = (
    pd.merge(labels, df, left_on="Attribute", right_on="x", how="inner")
      .drop(columns=["x", "Attribute"])
      .sort_values(by="ppi_corr", ascending=False)
)
print(joined_df)

# Save to latex table
joined_df.to_latex(
    buf = "../Figures/7_n0MME.tex", 
    index=False, 
    float_format="{:.3f}".format, 
    header=["Label", "PPI correlation", "Effective sample size"],
    caption="Effective sample size for each attribute in the Moral Machine experiment",
    label="tab:n0MME",
    column_format="lcc"
)

                                     Label  ppi_corr     n0
8  Preference for inaction vs intervention     0.353  11275
7        Sparing pedestrians vs passengers     0.314  10986
6           Sparing the lawful vs unlawful     0.309  10950
5             Sparing the fit vs the large     0.283  10786
4                     Sparing women vs men     0.263  10670
1        Sparing high status vs low status     0.230  10504
3                 Sparing the young vs old     0.198  10371
2         Sparing more characters vs fewer     0.192  10347
0                Sparing humans vs animals     0.049  10022


### PPI Power Analysis

In [83]:
df = pd.read_csv("../Data/3_gpt4turbo_wp_20241118.csv.gz", 
                 dtype={'UserID':'string', 'ResponseID':'string'})

Covs = ['PedPed', 'Barrier', 'CrossingSignal', 'NumberOfCharacters',
        'DiffNumberOFCharacters', 'LeftHand', 'Man', 'Woman', 'Pregnant',
        'Stroller', 'OldMan', 'OldWoman', 'Boy', 'Girl', 'Homeless',
        'LargeWoman', 'LargeMan', 'Criminal', 'MaleExecutive',
        'FemaleExecutive', 'FemaleAthlete', 'MaleAthlete', 'FemaleDoctor',
        'MaleDoctor', 'Dog', 'Cat', 
        'Intervention'
        ]

In [105]:
from scipy.optimize import brentq
from scipy.stats import norm, foldnorm
from ppi_py import ppi_ols_pointestimate, ppi_ols_ci

def effective_n_from_effect_size(delta, sigma, power, alpha=0.05):
    assert 0 < power < 1, "Power must be between 0 and 1"
    assert 0 < alpha < 1, "Level must be between 0 and 1"
    assert 0 != delta, "Effect size must be non-zero"
    assert power > alpha, "Power must be greater than alpha level"
    q = norm.ppf(1 - alpha / 2)
    beta = 1 - power

    def f(mu):
        return foldnorm.cdf(q, c=mu) - beta

    a = 0
    b = q - norm.ppf(beta)
    mu_star = brentq(f, a, b)
    d = delta/sigma
    effective_n = (mu_star/d)**2
    return round(effective_n)

In [103]:
# Define sample sizes
n = 10**4
N = 10**5

# Sample decisions for human subjects sample
n_data = df.sample(n=n, replace = False, random_state=20250117)

# Drop these decisions from the full dataset
df_remaining = df.drop(n_data.index)

# Sample decisions for silicon sample
N_data = df_remaining.sample(n=N, replace = False, random_state=2024)

# calculate weights for conjoint experiment
n_data.loc[:,"weights"] = pf.calcWeightsTheoretical(n_data)
N_data.loc[:,"weights"] = pf.calcWeightsTheoretical(N_data)

# drop rows with missing values on dependent variable
n_dd = n_data.dropna(subset="gpt4turbo_wp_Saved")
N_dd = N_data.dropna(subset="gpt4turbo_wp_Saved")

# if X=1 characters die if AV serves, if X=0 characters if AV stays
n_X = n_dd["Intervention"]               
N_X = N_dd["Intervention"]

# add intercept
n_X = np.column_stack((np.ones(n_X.shape[0]), n_X))
N_X = np.column_stack((np.ones(N_X.shape[0]), N_X))

# gold standard data
n_Y_human   = n_dd["Saved"].to_numpy()              # observed outcomes
n_Y_silicon = n_dd["gpt4turbo_wp_Saved"].to_numpy() # predicted outcomes
n_weights = n_dd["weights"].to_numpy()              # define weights

# unlabeled data
N_Y_silicon = N_dd["gpt4turbo_wp_Saved"].to_numpy() # predicted outcomes
N_weights = N_dd["weights"].to_numpy()              # define weights

In [107]:
# specify the position of the coefficient of interest in the vector of coefficients
coord   = 1

# specify effect under the null hypothesis
theta_0 = 0

# using ppi function to calculate point estimates based only on human subjects (lambda=0)
theta_1 = ppi_ols_pointestimate(
    X=n_X, 
    Y=n_Y_human, 
    Yhat=n_Y_silicon, 
    X_unlabeled=N_X, 
    Yhat_unlabeled=N_Y_silicon, 
    w=n_weights, 
    w_unlabeled=N_weights, 
    lam=0
)[coord]

# calculate effect 
delta = theta_1 - theta_0

# compute confidence interval for human subjects estimator
lower, upper = ppi_ols_ci(
    X=n_X, 
    Y=n_Y_human, 
    Yhat=n_Y_silicon, 
    X_unlabeled=N_X, 
    Yhat_unlabeled=N_Y_silicon, 
    w=n_weights, 
    w_unlabeled=N_weights, 
    lam = 0,
    alpha = 0.05
)
# zscore for two-sided test
z = stats.norm.ppf(0.975)

# compute standard error for human subjects estimator
se = (upper[coord] - lower[coord])/2/z

# 
sigma = se * n_X.shape[0]**0.5


effective_n = effective_n_from_effect_size(delta=delta, sigma=sigma, power=0.9, alpha=0.05)
effective_n

1846

In [72]:
powerful_pair = ppi_py.ppi_power(
    ppi_corr = 0.352677, 
    cost_X=0, 
    cost_Y=1, 
    cost_Yhat=0.001, 
    budget=10000
)
powerful_pair

{'n': 9882,
 'N': 107898,
 'cost': 9999.78,
 'effective_n': 11153,
 'ppi_corr': 0.352677}

In [24]:
cheapest_pair = ppi_py.ppi_power(
    ppi_corr = 0.352677, 
    cost_X = 0, 
    cost_Y = 1, 
    cost_Yhat = 0.01, 
    effective_n=1163962,
)
cheapest_pair


{'n': 1057600,
 'N': 2928433,
 'cost': 1097460.33,
 'effective_n': 1163962,
 'ppi_corr': 0.352677}